In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from operator import itemgetter
from random_traj import randaniso, randconv, multrandaniso, multrandconv, randsamp, multrandsamp, randall, multrandall
from random_traj import randanisorot, multrandanisorot
from ellipsoid import maxtraj, mvee, plot_mvee, plot_mveeoverl, rotmat, rotmat2, plot_mveeoverl2
from ellipsoid import ellipsax, MSDS, andiff, plot_anisoMSDS, plot_anisodiff
import random
import numpy as np
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D
from ellipsoid import extrema, enclosed_MSD

pi = np.pi
sin = np.sin
cos = np.cos

In [ ]:
# The purpose of this code is to continue work from _Final_Data_Analysis.  I will construct a code that will average
# data from multiple videos.  

#I am modifying this notebook further to allow the comparison of two datasets at different regions or ages.

In [12]:
# Type in datasets you would like to combine.  Adjust totvids to reflect the total number of videos. Also type in the
# umppx and fps in the conversion dictionary.

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.11, 4)
conversion[2] = (0.11, 4)
conversion[3] = (0.11, 4)
conversion[4] = (0.11, 4)
conversion[5] = (0.11, 4)

trajectory = dict()

trajectory[1] = np.genfromtxt('../sample_data/gopher_S2_R4_V1_10NPs_traj.csv',
            delimiter =",")
trajectory[2] = np.genfromtxt('../sample_data/gopher_S2_R4_V2_10NPs_traj.csv',
            delimiter =",")
trajectory[3] = np.genfromtxt('../sample_data/gopher_S2_R4_V3_10NPs_traj.csv',
            delimiter =",")
trajectory[4] = np.genfromtxt('../sample_data/gopher_S2_R4_V4_10NPs_traj.csv',
            delimiter =",")
trajectory[5] = np.genfromtxt('../sample_data/gopher_S2_R4_V5_10NPs_traj.csv',
            delimiter =",")

totvids = 5

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    #trajectory[num]=np.delete(trajectory[num],0,1)
    

In [13]:

final = dict()
tots = dict()

for num in range(1, totvids + 1):
    (final[num], tots[num]) = prettify(trajectory[num], 80, 80, conversion[num][0], conversion[num][1])
    

In [14]:
parts = dict()
lehi = dict()

for num in range(1, totvids + 1):
    parts[num] = tots[num]
    lehi[num] = final[num][1]
    counter = 1

    while counter < parts[num]:
        counter = counter + 1
        lehi[num] = np.append(lehi[num], final[num][counter], axis=0)

for num in range(2, totvids + 1):
    lehi[num][:,0] = lehi[num][:,0] + max(lehi[num-1][:, 0])        
        
nephi = lehi[1]
counter = 1

while counter < totvids:
    counter = counter + 1
    nephi = np.append(nephi, lehi[counter], axis=0)


In [15]:
# Now for a plotting function, as modified from Testing_Tracking_Software

# Creates an array 'particles' that contains the particle number at each frame.
particles = nephi[:, 0]
position = nephi[:, 5:5+4]
total = int(max(particles))
total1 = total + 1
path = dict()

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    path[num] = (position[min1:max1, :])

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# Plots individual trajectories
for num in range(1, total1):

    ax.plot(path[num][:, 0], path[num][:, 1], label='Particle {}'.format(num), linewidth=5)

axbox = ax.get_position()

legunit = max(nephi[:,0])
#ax.legend(loc=(0.85, 0.96-0.0273*legunit), prop={'size': 20})
ax.locator_params(nbins=6)

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)

ax.title.set_fontsize(35)
ax.tick_params(direction='out', pad=3)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))

dim = 200
plt.gca().set_xlim([-dim, dim])
plt.gca().set_ylim([-dim, dim])

# Save your figure
plt.savefig('{}.png'.format('test_1'), bbox_inches='tight')

plt.show()

In [16]:
nephi[:, 0]

array([   1.,    1.,    1., ...,  177.,  177.,  177.])

In [17]:
def plot_Mean1DMSDsorDeff(traj, n1, n2, n3, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(time[1][:], MMSD[:, 0], linewidth=10, label='2D')
    ax.plot(time[1][:], MMSD[:, 1], linewidth=10, label='1D x')
    ax.plot(time[1][:], MMSD[:, 2], linewidth=10, label='1D y')

    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.65, 0.05), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return MMSD



In [19]:
one = plot_Mean1DMSDsorDeff(nephi, 0, 15, 9, 0, 0, 'MSD (um^2)', 'gopher_S2_R4_177NPs_MSD', 21, 101, 4, 20)
plt.show()

In [21]:
one = plot_Mean1DMSDsorDeff(nephi, 0, 15, 17, 0, 1, 'Deff (um^2/s)', 'gopher_S2_R4_134NPs_Deff', 21, 3.5, 4, 0.5)
plt.show()

In [ ]:
nephi.shape[0]

In [ ]:
nephi[:, 0]